# Functions for VR analysis

In [13]:
import os

#Loading up necessary libraries
from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import precision_recall_fscore_support


#Loads up machine learning tools for LDA, LR, DT, and SVM
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import LeaveOneOut
from sklearn.cluster import KMeans

from sklearn.model_selection import cross_val_score
from sklearn.metrics import plot_confusion_matrix
import seaborn as sns


# File loader

In [14]:
def fileload (filename, choice):
    path = ('/Users/binhnguyen/Documents/MATLAB/2. BIOPAC/')
    # filename = ('python_FS_P1.mat')
    
    if (choice == 1):
        keys = 'python_FS'        
    elif (choice == 2):
        keys = 'label'
    elif (choice == 3):
        keys = 'mi_val'
    elif (choice == 4):
        keys = 'pss_val'
    elif (choice == 5):
        keys = 'feat_names'

    file = (path + filename)
    feature_set = loadmat(file)
#     print (file)


    return feature_set[keys]


# Normalization

In [15]:
def normalization (x):
    normalized = (x-min(x))/(max(x)-min(x))
    return (normalized)

# Creating the label matrix

In [16]:

# Putting scores into range categories

def ranges_binary (labels):
    output = np.zeros ((len(labels),1))
    for i in range (0,len(labels)):
        if (labels [i] <16): # Low stress
            output [i] = 0
        elif (labels [i] >=16): # Moderate stress
            output [i] = 1
            
    return output

# ML Function

In [17]:
## Hyperplane graph

def hyperplane (clf,X,Y):
    w = clf.coef_[0]
    a = -w[0] / w[1]
    xx = np.linspace(-1, 3)
    yy = a * xx - (clf.intercept_[0]) / w[1]
    plt.figure(1, figsize=(4, 3))
    plt.clf()
    plt.plot(xx, yy, "k-") #********* This is the separator line ************

    plt.scatter(X[:, 0], X[:, 1], c=Y, zorder=10, cmap=plt.cm.Paired, edgecolors="k")
    plt.show()

## Confusion Matrix

def CM (y_test, y_pred,title):

    cm = confusion_matrix(y_test,y_pred)
    plt.figure()
    plt.title('Confusion matrix of the classifier')
    sns.heatmap(cm,annot=True, fmt=".1f")
    plt.xlabel('Predicted')
    plt.xticks()
    plt.ylabel('True')
    plt.ioff()
    plt.show()


## SVM

def SVM_prediction(X_train, y_train, X_test,y_test):
    
    clf = SVC(gamma='scale', class_weight='balanced')
#     clf = svm.SVC(C=1.0, kernel='linear', max_iter=-1, decision_function_shape='ovr') 
    clf.fit(X_train, y_train) # Create SVM model based on default settings
    y_pred = clf.predict (X_test) # SVM prediction for test values
    acc = accuracy_score(y_test, y_pred) #Accuracy of the model
    CM(y_test, y_pred,'SVM')
    print ('The accuracy of SVM is %f%%' %(acc*100)) #Prints accuracy
    
    print ("Precision/Recall/F1/Support:")
    print (precision_recall_fscore_support(y_test, y_pred, average='macro'))
    
    return acc

## DT

def DT_prediction(X_train, y_train, X_test,y_test):
    clf = DecisionTreeClassifier(class_weight='balanced', ccp_alpha=0.0) 
    clf.fit(X_train, y_train) # Create DT model based on default setting
    y_pred = clf.predict (X_test) # DT prediction for test values
    acc = accuracy_score(y_test, y_pred) #Accuracy of the model
    CM(y_test, y_pred,'DT')
    print ('The accuracy of DT is %f%%' %(acc*100)) #Prints accuracy
    
    print ("Precision/Recall/F1/Support:")
    print (precision_recall_fscore_support(y_test, y_pred, average='macro'))
    
    return acc

## LR

def LR_prediction(X_train, y_train, X_test,y_test):
    clf = LogisticRegression() # Load-up of LR ML methodology using default setting
    clf.fit(X_train, y_train) # Create LR model based on default setting
    y_pred = clf.predict (X_test) # LR prediction for test values
    acc = accuracy_score(y_test, y_pred) #Accuracy of the model
    CM(y_test, y_pred,'LR')
    print ('The accuracy of LR is %f%%' %(acc*100)) #Prints accuracy
    return acc

# Visualization of Test and Training sets

In [18]:

# Plot histograms of labels
def histo_labels (par_num,feature_matrix,y_train,y_test):
    print ('Participant: %d \nTotal samples available: %d\
            \nNumber of training samples: %d \nNumber of testing samples: %d \n'\
           %(par_num, len(feature_matrix),len(y_train),len(y_test)))

    plt.figure ()
    plt.hist (y_train)
#     plt.xticks(range(int (np.max (y_train)+2)))
    plt.xlabel ('Brief MIOS score')
    plt.ylabel ('Frequency')
    plt.title ('Training set Histogram')

    plt.figure ()
    plt.hist (y_test)
#     plt.xticks(range(int (np.max (y_train)+2)))
    plt.xlabel ('Brief MIOS score')
    plt.ylabel ('Frequency')
    plt.title ('Testing set Histogram')
    

In [19]:

# Plot histograms of labels
def histo_labels_pss (par_num,feature_matrix,y_train,y_test):
    print ('Participant: %d \nTotal samples available: %d\
            \nNumber of training samples: %d \nNumber of testing samples: %d \n'\
           %(par_num, len(feature_matrix),len(y_train),len(y_test)))

    plt.figure ()
    plt.hist (y_train)
#     plt.xticks(range(int (np.max (y_train)+2)))
    plt.xlabel ('PSS score')
    plt.ylabel ('Frequency')
    plt.title ('Training set Histogram')

    plt.figure ()
    plt.hist (y_test)
#     plt.xticks(range(int (np.max (y_train)+2)))
    plt.xlabel ('PSS score')
    plt.ylabel ('Frequency')

    plt.title ('Test set Histogram')
    

# Arbitrary clustering of labels
Cut-off of 0-10, 10-20, 20-30

In [20]:
def cluster_arbitrary_3 (label, n_participants, cutoff):
    hold2 =  np.hstack ((label))

    c1 = (len ([i for i in hold2 if ((i <= cutoff[0]))]))
    c2 = (len ([i for i in hold2 if ((i <= cutoff[1])and(i>cutoff[0]))]))
    c3 = (len ([i for i in hold2 if ((i > cutoff[1]))]))


    plt.figure()
    plt.hist (label)
    plt.title ("Histogram of labels of %d participants" %n_participants)
    plt.ylabel ('Frequency')
    plt.xlabel ('Brief MIOS Score')

    # Hot encode to 3 clusters
    hold2 [hold2<cutoff[0]] = 1
    hold2 [(hold2>=cutoff[0]) & (hold2<=cutoff[1])] = 2
    hold2 [hold2>cutoff[1]] = 3

    print ('Length of Cluster 1: %d Cluster 2: %d Cluster 3: %d' %(c1,c2,c3))
    plt.figure ()
    plt.plot (hold2)
    plt.title ("Clustered label array of %d participants" %n_participants)
    plt.ylabel ('Score')
    plt.xlabel ('Sample')
    
    return (hold2)




# Arbitrary clustering of labels
Binary of above and below

In [21]:
def cluster_binary(label, n_participants, cutoff_binary):

    hold2 =  np.hstack ((label))
    c1 =  (len (hold2[hold2<cutoff_binary]))
    c2 = (len (hold2[hold2>cutoff_binary]))

    plt.figure()
    plt.hist (label)
    plt.title ("Histogram of labels of %d participants" %n_participants)
    plt.ylabel ('Frequency')
    plt.xlabel ('Brief MIOS Score')

    # Hot encode to 3 clusters
    hold2 [hold2<=cutoff_binary] = 1
    hold2 [hold2>cutoff_binary] = 2

    print ('Length of Cluster 1: %d Cluster 2: %d' %(c1,c2))
    plt.figure ()
    plt.plot (hold2)
    plt.title ("Clustered label array of %d participants" %n_participants)
    plt.ylabel ('Score')
    plt.xlabel ('Sample')
    
    return (hold2)

# Unstack clusters

In [22]:
def unstack_cluster (label):
    Y_hold1 = np.zeros (1)
    for i in label:    
        Y_hold1 = np.append (Y_hold1,i)
    Y_hold =  np.delete(Y_hold1, 0)
    return (Y_hold)

# Kmeans 

- Use the elbow method for optimal clusters

- Then assign the labels

https://www.geeksforgeeks.org/elbow-method-for-optimal-value-of-k-in-kmeans/


In [23]:
def kmeans_elbow (X):
    from sklearn.cluster import KMeans
    from sklearn import metrics
    from scipy.spatial.distance import cdist

    n_clusterz = 8

    distortions = []
    inertias = []
    mapping1 = {}
    mapping2 = {}
    K = range(1, n_clusterz)

    for k in K:
        # Building and fitting the model
        kmeanModel = KMeans(n_clusters=k).fit(X)
        kmeanModel.fit(X)

        distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_,
                                            'euclidean'), axis=1)) / X.shape[0])
        inertias.append(kmeanModel.inertia_)

        mapping1[k] = sum(np.min(cdist(X, kmeanModel.cluster_centers_,
                                       'euclidean'), axis=1)) / X.shape[0]
        mapping2[k] = kmeanModel.inertia_

    for key, val in mapping1.items():
        print(f'{key} : {val}')

    plt.plot(K, distortions, 'bx-')
    plt.xlabel('Values of K')
    plt.ylabel('Distortion')
    plt.title('The Elbow Method using Distortion')
    plt.show()


    for key, val in mapping2.items():
        print(f'{key} : {val}')

    plt.plot(K, inertias, 'bx-')
    plt.xlabel('Values of K')
    plt.ylabel('Inertia')
    plt.title('The Elbow Method using Inertia')
    

def kmeans_cluster_label(X,label,n_cluster):

    kmeans = KMeans(n_cluster, random_state=0).fit(X)

    Y_pred = kmeans.labels_
    
    if (n_cluster == 2):
        print ('Length of Cluster 1: %d Cluster 2: %d'\
               %(len (Y_pred [Y_pred == 0]),len (Y_pred [Y_pred == 1])))
        
    elif (n_cluster == 3):
        print ('Length of Cluster 1: %d Cluster 2: %d Cluster 3: %d'\
               %(len (Y_pred [Y_pred == 0]),len (Y_pred [Y_pred == 1]),len (Y_pred [Y_pred == 2])))
    
    elif (n_cluster == 4):
        print ('Length of Cluster 1: %d Cluster 2: %d Cluster 3: %d Cluster 4: %d'\
               %(len (Y_pred [Y_pred == 0]),len (Y_pred [Y_pred == 1]),len (Y_pred [Y_pred == 2]),len (Y_pred [Y_pred == 3])))

    plt.figure()
    plt.hist (label)
    plt.title ("Histogram of labels of %d participants" %n_participants)
    plt.ylabel ('Frequency')
    plt.xlabel ('Brief MIOS Score')

    plt.figure()
    plt.plot (kmeans.labels_)
    plt.title ("kmeans unsupervised label array")
    plt.ylabel ('Cluster')
    plt.xlabel ('Sample')
    
    return (Y_pred)


# Validation of SSA

In [24]:
def validation (n_samples_per_par, X, Y,ft_stacked):
    # Validation for SSA - normal analysis

    par_num = 0
    start_train = n_samples_per_par [par_num]
    end_train = n_samples_per_par [par_num+1]

    X_test = X[start_train:end_train]
    y_test = Y[start_train:end_train]

    X_train, X_test, y_train, y_test = train_test_split(X_test, y_test, test_size=0.4, shuffle=True, random_state=70)
    histo_labels (par_num+1,ft_stacked,y_train,y_test)
    DT_prediction (X_train, y_train, X_test,y_test)